<a href="https://colab.research.google.com/github/alayuala/114-1PL.repo/blob/main/WEEK9_%E5%8D%88%E9%A4%90_%E8%81%9A%E6%9C%83%E6%B1%BA%E7%AD%96%E5%99%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **午餐/聚會決策器（帶條件過濾）（作業五）**
* 目標：從 Sheet 讀餐廳清單（價位/距離/是否營業）→ 篩選 → 隨機推薦 3 家 → 寫建議清單。
* AI 點子：把 3 家優缺點壓成 100 字說明，幫助快速決策。


In [13]:
# --- ================================== ---
# 🚀 午餐決策器 Pro (Folium + AI 版)
# 最終合併版 (20 筆餐廳 + gemini-2.5-flash)
# --- ================================== ---

# --- 1. 安裝所有必要的套件 ---
print("⏳ 正在安裝必要的套件 (gradio, folium, gspread...)...")
!pip install -q gradio folium gspread gspread_dataframe pandas google-auth google-auth-oauthlib google-auth-httplib2 google-generativeai

print("✅ 所有套件安裝完成！")

# --- 2. 載入所有函式庫 ---
import os
import pandas as pd
import gspread
import gradio as gr
import folium
import google.generativeai as genai
from google.colab import auth, userdata
from google.auth import default
from gspread_dataframe import set_with_dataframe, get_as_dataframe

print("✅ 函式庫載入完成。")

# --- 3. Google 認證 與 API 金鑰設定 ---

print("⏳ 正在進行 Google 認證...")
# (A) Google Sheet 認證 (Colab)
try:
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)
    print("✅ Colab 帳號認證成功 (Google Sheet)。")
except Exception as e:
    print(f"❌ Colab 認證失敗：{e}")

# (B) Gemini API 金鑰設定 (重要！)
try:
    GEMINI_API_KEY = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=GEMINI_API_KEY)

    # ***【已修正】***
    # 使用您指定的 'gemini-2.5-flash' 模型
    model = genai.GenerativeModel('gemini-2.5-flash')
    print("✅ Gemini AI 模型 (gemini-2.5-flash) 初始化成功。")

except Exception as e:
    print(f"❌ 找不到或無法設定 Gemini API 金鑰！")
    print(f"錯誤訊息：{e}")
    print("請確保您已在 Colab Secrets 中將金鑰命名為 'GOOGLE_API_KEY'。")
    model = None

# --- 4. 建立與寫入 Google Sheet 資料庫 (共 20 筆) ---

SHEET_URL = "https://docs.google.com/spreadsheets/d/1CGq3MnGLzwuVnHdOhBhSbSPE_4x7nHLQMKVYIIZI3Vg/edit?usp=sharing"
WORKSHEET_NAME = "餐廳列表"

print(f"⏳ 正在建立並寫入 20 筆範例資料到 Google Sheet '{WORKSHEET_NAME}'...")

# ***【已更新】***：
# 這裡是包含 20 筆餐廳的完整清單
complete_data = [
    # 原始 10 筆
    {'店家名稱': '阜杭豆漿', '地址': '台北市中正區忠孝東路一段108號2樓', '價位': '$', '距離': '中', '是否營業': '是', '特色描述': '米其林推薦的超人氣厚燒餅油條', 'lat': 25.0456, 'lng': 121.5228},
    {'店家名稱': '鼎泰豐 (信義本店)', '地址': '台北市大安區信義路二段194號', '價位': '$$$', '距離': '中', '是否營業': '是', '特色描述': '世界知名的小籠包，服務一流', 'lat': 25.0336, 'lng': 121.5308},
    {'店家名稱': '永康牛肉麵', '地址': '台北市大安區金山南路二段31巷17號', '價位': '$$', '距離': '中', '是否營業': '是', '特色描述': '紅燒與清燉湯頭皆為經典', 'lat': 25.0326, 'lng': 121.5292},
    {'店家名稱': '阿宗麵線 (西門町)', '地址': '台北市萬華區峨眉街8-1號', '價位': '$', '距離': '長', '是否營業': '是', '特色描述': '西門町指標美食，站著吃', 'lat': 25.0435, 'lng': 121.5073},
    {'店家名稱': '樂業麵線', '地址': '台北市大安區樂業街87號', '價位': '$', '距離': '短', '是否營業': '是', '特色描述': 'CP值超高的大腸麵線，料多實在', 'lat': 25.0204, 'lng': 121.5525},
    {'店家名稱': 'J&M 咖啡', '地址': '台北市大安區瑞安街199號', '價位': '$$', '距離': '短', '是否營業': '否', '特色描述': '安靜巷弄的專業手沖咖啡', 'lat': 25.0280, 'lng': 121.5420},
    {'店家名稱': '通化街蚵仔煎', '地址': '台北市大安區臨江街104號', '價位': '$', '距離': '中', '是否營業': '是', '特色描述': '臨江街夜市內的經典小吃', 'lat': 25.0315, 'lng': 121.5545},
    {'店家名稱': 'My灶', '地址': '台北市中山區松江路100巷9-1號', '價位': '$$$', '距離': '長', '是否營業': '是', '特色描述': '米其林一星的精緻台菜，滷肉飯必點', 'lat': 25.0504, 'lng': 121.5298},
    {'店家名稱': '上引水產', '地址': '台北市中山區民族東路410巷2弄18號', '價位': '$$$', '距離': '長', '是否營業': '是', '特色描述': '大型水產市集，可立吞壽司或吃火鍋', 'lat': 25.0683, 'lng': 121.5310},
    {'店家名稱': '好丘 Good Cho\'s (信義店)', '地址': '台北市信義區松勤街54號', '價位': '$$', '距離': '中', '是否營業': '是', '特色描述': '四四南村內的貝果專賣店，口味多元', 'lat': 25.0337, 'lng': 121.5599},
    # 新增 10 筆
    {'店家名稱': '金峰魯肉飯', '地址': '台北市中正區羅斯福路一段10號', '價位': '$', '距離': '中', '是否營業': '是', '特色描述': '南門市場旁的人氣滷肉飯，湯品也推薦', 'lat': 25.0342, 'lng': 121.5169},
    {'店家名稱': '詹記麻辣火鍋 (敦南店)', '地址': '台北市大安區和平東路三段60號', '價位': '$$$', '距離': '長', '是否營業': '是', '特色描述': '一位難求的經典台式麻辣鍋，鴨血必吃', 'lat': 25.0244, 'lng': 121.5491},
    {'店家名稱': 'Smith & Wollensky 台北', '地址': '台北市信義區松智路17號47樓', '價位': '$$$', '距離': '長', '是否營業': '是', '特色描述': '股神巴菲特愛店，高空景觀牛排館', 'lat': 25.0338, 'lng': 121.5645},
    {'店家名稱': '杭州小籠湯包 (本店)', '地址': '台北市大安區杭州南路二段19號', '價位': '$$', '距離': '中', '是否營業': '是', '特色描述': '高CP值的平價版鼎泰豐，絲瓜蝦仁湯包清甜', 'lat': 25.0315, 'lng': 121.5283},
    {'店家名稱': '藍家割包', '地址': '台北市中正區羅斯福路三段316巷8弄3號', '價位': '$', '距離': '長', '是否營業': '是', '特色描述': '公館排隊美食，肥瘦可選的美味割包', 'lat': 25.0163, 'lng': 121.5327},
    {'店家名稱': '興波咖啡 (Simple Kaffa)', '地址': '台北市中正區忠孝東路二段27號', '價位': '$$', '距離': '中', '是否營業': '是', '特色描述': '世界冠軍咖啡師吳則霖的旗艦店', 'lat': 25.0431, 'lng': 121.5255},
    {'店家名稱': '劉芋仔蛋黃芋餅', '地址': '台北市大同區寧夏路', '價位': '$', '距離': '長', '是否營業': '是', '特色描述': '寧夏夜市必吃，香酥芋丸包鹹蛋黃', 'lat': 25.0592, 'lng': 121.5133},
    {'店家名稱': '餵我早餐 (大安店)', '地址': '台北市大安區建國南路二段11巷28號', '價位': '$$', '距離': '短', '是否營業': '否', '特色描述': '人氣早午餐，The Whale 系列', 'lat': 25.0298, 'lng': 121.5375},
    {'店家名稱': '廖家牛肉麵', '地址': '台北市大安區金華街98號', '價位': '$$', '距離': '中', '是否營業': '是', '特色描述': '清燉湯頭的代表，湯鮮肉嫩', 'lat': 25.0305, 'lng': 121.5307},
    {'店家名稱': '添好運 (台北車站店)', '地址': '台北市中正區忠孝西路一段36號1樓', '價位': '$$', '距離': '中', '是否營業': '是', '特色描述': '米其林一星的平價港點，酥皮焗叉燒包必點', 'lat': 25.0479, 'lng': 121.5170}
]
df_restaurants = pd.DataFrame(complete_data)

try:
    gsheets = gc.open_by_url(SHEET_URL)

    try:
        worksheet = gsheets.worksheet(WORKSHET_NAME)
    except gspread.exceptions.WorksheetNotFound:
        print(f"找不到工作表 '{WORKSHEET_NAME}'，正在自動建立...")
        worksheet = gsheets.add_worksheet(title=WORKSHEET_NAME, rows="100", cols="20")

    worksheet.clear()

    # 確保 lat/lng 欄位是 float 格式
    df_restaurants['lat'] = df_restaurants['lat'].astype(float)
    df_restaurants['lng'] = df_restaurants['lng'].astype(float)

    set_with_dataframe(worksheet, df_restaurants)

    print(f"✅ 成功將 {len(df_restaurants)} 筆資料寫入 Google Sheet！")

except Exception as e:
    print(f"❌ 寫入 Google Sheet 時發生錯誤：{e}")
    print("請檢查：1. SHEET_URL是否正確 2. 您是否已將 Sheet 分享權限設為「可編輯」。")


# --- 5. 讀取資料並準備篩選器 (Phase 3 的準備) ---
# (我們直接使用剛剛建立的 df_restaurants，不再重新讀取，更有效率)

try:
    df_restaurants['lat'] = pd.to_numeric(df_restaurants['lat'])
    df_restaurants['lng'] = pd.to_numeric(df_restaurants['lng'])
    df_restaurants = df_restaurants.dropna(subset=['店家名稱', 'lat', 'lng'])

    # 預先抓取所有唯一的篩選條件
    unique_prices = df_restaurants['價位'].unique().tolist()
    unique_distances = df_restaurants['距離'].unique().tolist()
    print(f"✅ 資料庫已準備就緒，共 {len(df_restaurants)} 筆餐廳。")

except Exception as e:
    print(f"❌ 讀取資料失敗：{e}")
    df_restaurants = pd.DataFrame(columns=['店家名稱', '地址', '價位', '距離', '是否營業', '特色描述', 'lat', 'lng'])
    unique_prices = ['$']
    unique_distances = ['短']

# --- 6. 核心功能函數 (Backend) ---

def filter_and_recommend(price_filter, distance_filter, is_available):
    """根據使用者輸入篩選並隨機推薦 3 家餐廳"""
    filtered_df = df_restaurants.copy()
    if price_filter:
        filtered_df = filtered_df[filtered_df['價位'].isin(price_filter)]
    if distance_filter:
        filtered_df = filtered_df[filtered_df['距離'].isin(distance_filter)]
    if is_available:
        filtered_df = filtered_df[filtered_df['是否營業'] == '是']
    if filtered_df.empty:
        return pd.DataFrame()
    n_recommend = min(3, len(filtered_df))
    return filtered_df.sample(n=n_recommend, replace=False)

def generate_ai_summary(recommended_df):
    """呼叫 Gemini AI 產生決策建議摘要"""
    if recommended_df.empty:
        return "沒有符合條件的餐廳，AI 顧問今天只好提早下班了。"
    if not model:
        return "AI 服務未啟動 (請檢查您的 'GOOGLE_API_KEY' 是否已在 Colab Secrets 中設定)。"

    ai_input_list = [
        f"{row['店家名稱']} (價位{row['價位']}, 距離{row['距離']}, 特色: {row['特色描述']})"
        for _, row in recommended_df.iterrows()
    ]
    ai_input = "、".join(ai_input_list)

    system_prompt = (
        "您是一位幽默風趣的美食決策顧問。您的任務是根據提供的2~3家餐廳清單，"
        "分析其優缺點（如價格、距離、特色）並總結，幫助使用者快速做出決定。"
        "請以輕鬆且誘人的口吻，用 **繁體中文** 撰寫一個約 **100 字** 的決策建議摘要。"
        "請不要使用 Markdown 標題或編號清單，直接給一段流暢的建議。"
    )
    user_query = f"這是今天的午餐候選清單：{ai_input}。請綜合考量，給我一份快速決策建議！"

    try:
        # ***【已修正】***
        # 使用 'gemini-2.5-flash'，它支援 system_prompt (系統提示詞)
        response = model.generate_content(
            [system_prompt, user_query],
            generation_config=genai.types.GenerationConfig(temperature=0.8)
        )
        return response.text.strip()
    except Exception as e:
        return f"AI 摘要產生失敗：{e}"

def create_folium_map(recommended_df):
    """根據推薦的餐廳建立 Folium 地圖"""
    taipei_center = [25.0479, 121.5319]
    if recommended_df.empty:
        m = folium.Map(location=taipei_center, zoom_start=13)
    else:
        map_center = [recommended_df['lat'].mean(), recommended_df['lng'].mean()]
        m = folium.Map(location=map_center, zoom_start=15)
        for _, row in recommended_df.iterrows():
            folium.Marker(
                location=[row['lat'], row['lng']],
                popup=folium.Popup(f"<b>{row['地址']}</b><br>{row['特色描述']}", max_width=200),
                tooltip=f"<b>{row['店家名稱']}</b> ({row['價位']})",
                icon=folium.Icon(color="red", icon="cutlery", prefix='fa')
            ).add_to(m)
    return m._repr_html_()

# --- 7. Gradio 介面主函數 (Controller) ---

def get_recommendations(price, distance, open_now):
    """Gradio 按鈕點擊後的主函數"""
    recommended_df = filter_and_recommend(price, distance, open_now)
    map_html = create_folium_map(recommended_df)
    ai_summary = generate_ai_summary(recommended_df)

    if recommended_df.empty:
        display_df = pd.DataFrame(columns=["餐廳名稱", "價位", "距離", "特色描述"])
    else:
        display_df = recommended_df[['店家名稱', '價位', '距離', '特色描述']].reset_index(drop=True)

    return map_html, ai_summary, display_df

# --- 8. Gradio 介面配置 (Frontend) ---

print("⏳ 正在啟動 Gradio 應用程式...")

with gr.Blocks(theme=gr.themes.Soft(), title="午餐決策器 Pro") as demo:
    gr.Markdown(
        """
        # 🍽️ 午餐/聚會決策器 Pro (Folium + AI 版)
        ### 從您的 Google Sheet 清單中篩選，隨機推薦 3 家，並由 AI 協助您快速決策！
        """
    )
    with gr.Row():
        # --- 左側：篩選器 ---
        with gr.Column(scale=1, min_width=300):
            gr.Markdown("## 1. 篩選您的需求")
            price_select = gr.CheckboxGroup(
                choices=unique_prices, label="💰 價位 (可複選)", value=unique_prices
            )
            distance_select = gr.CheckboxGroup(
                choices=unique_distances, label="🏃 距離 (可複選)", value=unique_distances
            )
            available_toggle = gr.Checkbox(
                label="✅ 只看目前營業 (僅顯示 '是')", value=True
            )
            recommend_btn = gr.Button("🚀 幫我決定！", variant="primary")

        # --- 右側：輸出結果 (使用 Tab 區分) ---
        with gr.Column(scale=3):
            gr.Markdown("## 2. 推薦結果")
            with gr.Tabs():
                # Tab 1: Folium 地圖
                with gr.TabItem("📍 推薦地圖", id=0):
                    map_output = gr.HTML(label="餐廳位置地圖")

                # Tab 2: AI 決策
                with gr.TabItem("🤖 AI 決策顧問", id=1):
                    summary_output = gr.Textbox(
                        label="AI 決策摘要", lines=5, interactive=False
                    )

                # Tab 3: 餐廳列表
                with gr.TabItem("📋 餐廳列表", id=2):
                    dataframe_output = gr.DataFrame(
                        headers=["餐廳名稱", "價位", "距離", "特色描述"],
                        datatype=["str", "str", "str", "str"],
                        label="隨機推薦清單"
                    )

    # --- 9. 綁定按鈕與函數 ---
    recommend_btn.click(
        fn=get_recommendations,
        inputs=[price_select, distance_select, available_toggle],
        outputs=[map_output, summary_output, dataframe_output]
    )

# --- 10. 啟動 Gradio 應用 ---
demo.launch(debug=True, inbrowser=True)

⏳ 正在安裝必要的套件 (gradio, folium, gspread...)...
✅ 所有套件安裝完成！
✅ 函式庫載入完成。
⏳ 正在進行 Google 認證...
✅ Colab 帳號認證成功 (Google Sheet)。
✅ Gemini AI 模型 (gemini-2.5-flash) 初始化成功。
⏳ 正在建立並寫入 20 筆範例資料到 Google Sheet '餐廳列表'...
❌ 寫入 Google Sheet 時發生錯誤：name 'WORKSHET_NAME' is not defined
請檢查：1. SHEET_URL是否正確 2. 您是否已將 Sheet 分享權限設為「可編輯」。
✅ 資料庫已準備就緒，共 20 筆餐廳。
⏳ 正在啟動 Gradio 應用程式...
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://e97b8f3951ea827ab6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://e97b8f3951ea827ab6.gradio.live
